# Quality assurance with MNE report

Let's say we want to analyze 100 subjects.

How do we do quality assurancy in a scalable manner?

## MNE report

In [50]:
from mne.report import Report

A report contains:
    * Figures
    * Images
    * Custom HTML
    * Sliders

First, let us generate some figures for the report.

In [51]:
%matplotlib inline

import mne
from mne.datasets import sample

mne.viz.set_browser_backend('matplotlib')

data_path = sample.data_path()
raw_fname = data_path / 'MEG' / 'sample' / 'sample_audvis_raw.fif'
raw = mne.io.read_raw_fif(raw_fname, preload=True)

Opening raw data file /Users/mainak/mne_data/MNE-sample-data/MEG/sample/sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...


Now let's pretend this data came from 3 different subjects

In [52]:
raw1 = raw.copy().crop(0, 20)
raw2 = raw.copy().crop(20, 40)
raw3 = raw.copy().crop(40, 60)

raw1.save(data_path / 'MEG' / 'sample' / 'sub-01_raw.fif', overwrite=True)
raw2.save(data_path / 'MEG' / 'sample' / 'sub-02_raw.fif', overwrite=True)
raw3.save(data_path / 'MEG' / 'sample' / 'sub-03_raw.fif', overwrite=True)

Overwriting existing file.
Writing /Users/mainak/mne_data/MNE-sample-data/MEG/sample/sub-01_raw.fif
Closing /Users/mainak/mne_data/MNE-sample-data/MEG/sample/sub-01_raw.fif
[done]
Overwriting existing file.
Writing /Users/mainak/mne_data/MNE-sample-data/MEG/sample/sub-02_raw.fif
Closing /Users/mainak/mne_data/MNE-sample-data/MEG/sample/sub-02_raw.fif
[done]
Overwriting existing file.
Writing /Users/mainak/mne_data/MNE-sample-data/MEG/sample/sub-03_raw.fif
Closing /Users/mainak/mne_data/MNE-sample-data/MEG/sample/sub-03_raw.fif
[done]


Now, we can have a function to go from raw to evoked

In [53]:
event_id = {'Auditory/Left': 3, 'Auditory/Right': 4}

def raw_to_evoked(raw_fname, tmin=-0.1, tmax=0.5):
    
    raw = mne.io.read_raw_fif(data_path / 'MEG' / 'sample' / raw_fname, preload=True)
    fig1 = raw.plot();
    raw.filter(0, 40.)
    
    events = mne.find_events(raw, stim_channel='STI 014')
    epochs = mne.Epochs(raw, events, event_id, tmin, tmax)
    fig2 = epochs.plot();
    
    evoked_l = epochs['Left'].average();
    fig3 = evoked_l.plot_topomap()
    fig4 = evoked_l.plot();
    
    return [fig1, fig2, fig3, fig4]

Now, we can get all the figure handles:

In [54]:
%%capture
figs = raw_to_evoked('sub-01_raw.fif')

Now comes the actual report

In [55]:
rep = Report()

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : bootstrap-table/bootstrap-table.min.js
Embedding : bootstrap-table/bootstrap-table.min.css
Embedding : bootstrap-table/bootstrap-table-copy-rows.min.js
Embedding : bootstrap-table/bootstrap-table-export.min.js
Embedding : bootstrap-table/tableExport.min.js
Embedding : bootstrap-icons/bootstrap-icons.mne.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css


In [56]:
rep.add_figure?

In [58]:
captions = ['Raw', 'Epochs', 'Topomap', 'Butterfly']
for fig, caption in zip(figs, captions):
    rep.add_figure(fig, caption=caption, title=caption)
rep.save('report_raw_to_evoked.html', overwrite=True)

Overwriting existing file.
Saving report to : /Users/mainak/Desktop/github_repos/mne-workshop-mit/report_raw_to_evoked.html


'/Users/mainak/Desktop/github_repos/mne-workshop-mit/report_raw_to_evoked.html'

The report can be found [here](report_raw_to_evoked.html)

We can go even more fancy. Let's try to process all the three subjects.

In [60]:
%%capture
rep = Report()
for idx, r in enumerate(['sub-01_raw.fif', 'sub-02_raw.fif', 'sub-03_raw.fif']):
    figs = raw_to_evoked(r)
    for fig, caption in zip(figs, captions):
        rep.add_figure(fig, caption=caption, title=caption, section='Subject %02d' % idx)
rep.save('report_raw_to_evoked.html', overwrite=True)

There are tabs for each subject!

Parallel processing
-------------------

In [61]:
def raw_to_evoked(raw_fname, tmin=-0.1, tmax=0.5):
    raw = mne.io.read_raw_fif(data_path / 'MEG' / 'sample' / raw_fname, preload=True)
    raw.filter(0, 40.)
    events = mne.find_events(raw, stim_channel='STI 014')
    epochs = mne.Epochs(raw, events, event_id, tmin, tmax)
    evoked_l = epochs['Left'].average();

from mne.parallel import parallel_func

fnames = ['sub-01_raw.fif', 'sub-02_raw.fif', 'sub-03_raw.fif']
parallel, myfunc, _ = parallel_func(raw_to_evoked, n_jobs=3)
parallel(myfunc(fname) for fname in fnames);

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Opening raw data file /Users/mainak/mne_data/MNE-sample-data/MEG/sample/sub-02_raw.fif...
Opening raw data file /Users/mainak/mne_data/MNE-sample-data/MEG/sample/sub-01_raw.fif...
Opening raw data file /Users/mainak/mne_data/MNE-sample-data/MEG/sample/sub-03_raw.fif...
    Read a total of 3 projection items:
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 49825 ... 61837 =     82.957 ...   102.956 secs
    Range : 37812 ... 49825 =     62.955 ...    82.957 secs
    Range : 25800 ... 37812 =     42.956 ...    62.955 secs
Ready.
Ready.
Ready.
Reading 0 ... 12013  =      0.000 ...    20.001 secs...
Reading 0 ... 12012  =      0.000 ...    20.000 secs...
Reading 0 ... 1201

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.1s


25 events found on stim channel STI 014
Event IDs: [ 1  2  3  4  5 32]
30 events found on stim channel STI 014
Event IDs: [ 1  2  3  4  5 32]
Not setting metadata
Not setting metadata
13 matching events found
11 matching events found
31 events found on stim channel STI 014
Event IDs: [ 1  2  3  4  5 32]
Not setting metadata
14 matching events found
Setting baseline interval to [-0.09989760657919393, 0.0] s
Applying baseline correction (mode: mean)
Setting baseline interval to [-0.09989760657919393, 0.0] s
Applying baseline correction (mode: mean)
Setting baseline interval to [-0.09989760657919393, 0.0] s
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 3)
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Created an SSP operator (subspace dimension = 3)
3 projection items activated
3 projection items activated


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    3.3s finished


BEM sliders
-----------

What else can you do? You can inspect quality of the BEM with sliders.

In [63]:
subjects_dir = data_path / 'subjects'

rep = Report()
rep.add_bem(title='bem', subject='sample', subjects_dir=subjects_dir, decim=36)
rep.save('report_bem.html', overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : bootstrap-table/bootstrap-table.min.js
Embedding : bootstrap-table/bootstrap-table.min.css
Embedding : bootstrap-table/bootstrap-table-copy-rows.min.js
Embedding : bootstrap-table/bootstrap-table-export.min.js
Embedding : bootstrap-table/tableExport.min.js
Embedding : bootstrap-icons/bootstrap-icons.mne.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Using surface: /Users/mainak/mne_data/MNE-sample-data/subjects/sample/bem/inner_skull.surf
Using surface: /Users/mainak/mne_data/MNE-sample-data/subjects/sample/bem/outer_skull.surf
Using surface: /Users/mainak/mne_data/MNE-sample-data/subjects/sample/bem/outer_skin.surf
Overwriting existing file.
Saving report to : /Users/mainak/Desktop/github_repos/mne-workshop-mit/report_bem.html


'/Users/mainak/Desktop/github_repos/mne-workshop-mit/report_bem.html'

Check out the report [here](report_bem.html)

Custom HTML
--------------------

We can even add custom htmls. For example, we can say:

In [33]:
html = f"""
<table class="table table-hover">
   <tr>
       <th>Meas time range</th>
       <th>Sampling freq</th>
   </tr>
   <tr>
       <td> {0:.2f} to {1:.2f} </td>
       <td> {2:.2f} </td>
   </tr>
</table>
"""

In [34]:
rep.add_html(html.format(raw.times[0], raw.times[-1], raw.info['sfreq']), title='Info table')
rep.save('report_bem.html', overwrite=True)

Overwriting existing file.
Saving report to : /Users/mainak/Desktop/github_repos/mne-workshop-mit/report_bem.html


'/Users/mainak/Desktop/github_repos/mne-workshop-mit/report_bem.html'

Here is the [report](report_bem.html).

Custom sliders
--------------

And we can make our own sliders

In [36]:
evoked.plot_topomap?

In [46]:
import matplotlib.pyplot as plt
fname = data_path / 'MEG' / 'sample' / 'sample_audvis-ave.fif'
evoked = mne.read_evokeds(fname, condition='Left Auditory',
                          baseline=(None, 0), verbose=False)

rep = Report()
figs = list()
times = evoked.times[::20]
for time in times:
    figs.append(evoked.plot_topomap(time, vlim=(-300, 300),
                                    res=100, show=False))
    plt.close(figs[-1])
rep.add_figure(figs, title='Evoked Response', caption=times)
rep.save('report_slider.html', overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : bootstrap-table/bootstrap-table.min.js
Embedding : bootstrap-table/bootstrap-table.min.css
Embedding : bootstrap-table/bootstrap-table-copy-rows.min.js
Embedding : bootstrap-table/bootstrap-table-export.min.js
Embedding : bootstrap-table/tableExport.min.js
Embedding : bootstrap-icons/bootstrap-icons.mne.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Overwriting existing file.
Saving report to : /Users/mainak/Desktop/github_repos/mne-workshop-mit/report_slider.html


'/Users/mainak/Desktop/github_repos/mne-workshop-mit/report_slider.html'

To learn more about quality assurance, check out [this paper](https://www.frontiersin.org/journals/neuroscience/articles/10.3389/fnins.2018.00530/full)

Exercise
--------

1) Can you think of creative ways to use the report for your own analysis?

<details>
<summary>
Here are some ideas:<br/><br/>

</summary>
- sections with subject names instead of preprocessing step<br/>
- custom html/javascript to get quality labels<br/>
- sliders to browse through the raw data<br/>
</details>